In [35]:
import requests
from contents_DB import connect_mongodb, read_from_mongodb

In [41]:
# Your KAKAO REST API KEY

KAKAO_REST_API_KEY = 'Your KAKAO REST API KEY' # Kakao REST API KEY 입력

In [148]:
# MongoDB 연결

collection = connect_mongodb('mydatabase','mycollection')

In [149]:
# DB의 일부를 추출(테스트용)

results_df = read_from_mongodb(collection)
results_df = results_df[:80]

In [101]:
import re

# 층, 호 정보 제거 함수

def extract_road_address(address):

    # 도로명 주소 패턴: '도로명' + '건물번호'
    pattern = r'([가-힣]+\s[가-힣]+\s[가-힣0-9\-]+(?:로|길)\s*\d+(-\d+)?)'  # 도로명과 번호 추출
    match = re.search(pattern, address)
    if match:
        return match.group(0)  # 매칭된 첫 번째 그룹 반환
    return address  # 매칭 실패 시 원본 반환

In [130]:
# address를 입력하면 좌표를 반환하는 함수

def coord_info(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': address
    }
    headers = {
    "Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"  # 실제 API 키를 넣어야 합니다
    }
    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    if result['documents']:
        address = result['documents'][0]
        return address['y'], address['x']
    else:
        print("해당 주소를 찾을 수 없습니다.")
        return None, None

In [128]:
import requests
import json
import folium

# 매장명과 주소를 통해 매장의 정보를 얻는 함수
def search_place_by_address(address, place_name):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = json.loads(response.text)
        if result['documents']:
            # 정확한 검색 결과 정보 반환
            for i in result['documents']:
                if i['place_name'] == place_name:
                    
                    return {
                        'place_name': i['place_name'], # 정확한 매장명 확인을 위함
                        'address_name': i['road_address_name'], # 도로명 주소
                        'x': float(i['x']),  # 경도
                        'y': float(i['y'])   # 위도
                        }
                else:
                    continue
            
    return None

In [150]:
# 지도 초기화
map = folium.Map(location=[37.5665, 126.9780], zoom_start=12)  # 서울 시청 기준

In [151]:
# 데이터프레임의 place name과 address를 map에 마킹하기

headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}

# 데이터프레임 순회하며 장소 확인 및 매칭
for idx, row in results_df.iterrows():
    place_name = row['restaurant name']
    print(place_name)
    address = extract_road_address(row['address'])
    print(address)

    # 주소로 좌표 변환 후 매장 이름과 주소 얻기
    result = search_place_by_address(address, place_name)

    if result:
        matched_name = result['place_name']
        matched_address = result['address_name']

        # 매장 이름과 주소를 지도에 마킹
        folium.Marker(
            location=[result['y'], result['x']],
            popup=f"{matched_name} ({matched_address})",
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(map)
        
    else:
        y, x = coord_info(address)
        keyword_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
        keyword_params = {"query": place_name, "x": x, "y": y, "radius": 100}
        keyword_response = requests.get(keyword_url, headers=headers, params=keyword_params)
        if keyword_response.status_code == 200:
            keyword_result = json.loads(keyword_response.text)
            if keyword_result['documents']:
                place_name = keyword_result['documents'][0]['place_name']
                print(f"검색성공 - {place_name}")
                folium.Marker(
                    location=[y, x],
                    popup=f"{place_name} ({address})",
                    icon=folium.Icon(color="blue", icon="info-sign")
                    ).add_to(map)
            else:
                print(f"찾지 못함 - {place_name}")

        else:
            print(f"응답 실패 - '{place_name}'")

print("저장 완료")

플랜온서면점
부산 부산진구 중앙대로680번가길 38
검색성공 - 플랜온 서면점
플랜온서면점
부산 부산진구 중앙대로680번가길 38
검색성공 - 플랜온 서면점
무화
부산 부산진구 서전로67번길 27-9
마카러버
부산 금정구 부곡온천천로 170
비아조
부산 강서구 식만로 164
이븐도우
부산 수영구 망미번영로38번길 10
검색성공 - 이븐도우 망미점
송정물총칼국수
부산 해운대구 송정중앙로6번길 184
금빛사진관
부산 부산진구 동성로 32
빵빵이와 옥지의 만물상
부산 부산진구 서면로 39
찾지 못함 - 빵빵이와 옥지의 만물상
로우앤스윗
부산 해운대구 우동1로38번가길 1
검색성공 - 로우앤스윗 해운대
사랑방다실
부산 부산진구 서전로57번길 4
서면미술관
부산 부산진구 동천로 58
저수지
부산 부산진구 동천로132번길 12-5
카페 레이어드
부산 해운대구 센텀남대로 35
검색성공 - 카페 레이어드 부산센텀점
슈슈크레페
부산 수영구 민락로6번길 28
롱드라이버스
부산 부산진구 서전로57번길 4
우드앤숲
양산시 백동길 160
듀플릿 광안점
부산 수영구 광남로 195-6
사랑방다실
부산 부산진구 서전로57번길 4
진미횟집
부산 수영구 광안해변로 207
검색성공 - 진미회집
국밥집
부산 남구 용호동 372-15
찾지 못함 - 국밥집
그린노마드
부산 수영구 좌수영로 161-10
퐁실퐁실 카스테라
양산시 물금읍 화합2길 8-1
찾지 못함 - 퐁실퐁실 카스테라
마쯔오카
부산 중구 중앙대로 2
검색성공 - 마쯔오카 롯데백화점광복점
버터럽
부산 부산진구 서전로37번길 20
템플 오브 템포
부산 부산진구 동성로 35
보노베리
부산 금정구 장전온천천로 67-1
알이즈웰
경남 김해시 김해대로2315번길 32
찾지 못함 - 알이즈웰
포디움다이브엠
부산 수영구 광남로 96
타타스베이글
부산 사하구 낙동대로 304
검색성공 - 타타스베이글 부산사하점
루트몰트
울산 북구 박상진6로 5
루트몰트
울산 북구 박상진6로 5
로우앤스윗
부산 해운대구 우동1로38번가길 1
검색성공 - 로우앤스윗 해운대


In [152]:
# HTML 파일로 저장
map.save("matched_places_map.html")
print("지도 생성 완료. 파일명 : matched_places_map.html")

지도 생성 완료. 파일명 : matched_places_map.html
